<a href="https://colab.research.google.com/github/Diya910/Deep_learning/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

Importing the Dependencies

In [4]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data-Collection

In [8]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [9]:
#setup kaggle credentitals as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary['username']
os.environ["KAGGLE_KEY"] = kaggle_dictionary['key']

In [10]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 66% 17.0M/25.7M [00:00<00:00, 54.2MB/s]
100% 25.7M/25.7M [00:00<00:00, 74.2MB/s]


In [11]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [13]:
#unzip the dataset file
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip:
  zip.extractall()
  print('Done')

Done


Loading the Data

In [14]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [15]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [16]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [17]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [18]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [19]:
#Split data into training and test data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [20]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


Data Preprocessing

In [22]:
#Tokenize text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [23]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

LSTM --> long short term memory

In [24]:
#build the model

model = Sequential()
model.add(Embedding(5000, output_dim=128, input_length=200))
model.add(LSTM(128,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
#compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Training the model

In [29]:
model.fit(X_train, Y_train, epochs=5, batch_size = 64, validation_split=0.2)

Epoch 1/5
 28/500 [>.............................] - ETA: 6:05 - loss: 0.1541 - accuracy: 0.9392

KeyboardInterrupt: 

# Model Evaluation

In [30]:
loss, accuracy = model.evaluate(X_test, Y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

313/313 [==============================] - 28s 89ms/step - loss: 0.3330 - accuracy: 0.8630
Test Loss: 0.33303916454315186
Test Accuracy: 0.8629999756813049


Building a predictive system

In [31]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences(review)
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

#Example usage


In [32]:
new_review = "This movie was amazing!"
sentiment = predict_sentiment(new_review)
print("Sentiment:", sentiment)

1/1 [==============================] - 0s 430ms/step
Sentiment: positive
